In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using Piccolo
using LinearAlgebra
using CairoMakie
using QuantumToolbox
using SparseArrays

const ⊗ = kron

# I. Rotations
-----

## Goals
- Geodesics
- Controllability of LTI systems
- Dynamical Lie algebras and reachability

_Comparing initialization and optimization when restricted to a feasible manifold, and how warm starts are enabled by direct control._

![Initialization comparison between direct and indirect optimization.](../assets/fig2_feas-manifold.png)

In [ ]:
U_goal = PAULIS.X
T = 50
Δt = 0.2

# U_goal = exp(-im * H_eff * T * Δt)
H_eff = im * log(U_goal / T / Δt)

In [ ]:
# Draw the geodesic.

**Exercise**
- What happens if there is a drift operator? $H(t) = \Delta \omega Z + u(t) X$.
- Bonus: What about for an _Embedded operator_?

## Controllability

- Quick check: What happens when we kick a system $x_{n+1} = A x_n + B u_n$?

\begin{equation}
\mathcal{C} = \begin{bmatrix}
    B & A B & A^2 B & \cdots & A^{n-1} B
\end{bmatrix}
\end{equation}

- Quick check: Why did we stop at $n-1$?

**Example**
- Test on a linear system in 2D. Recall our $F = ma$ system.

In [ ]:
function continuous_to_discrete(A, B, h)
    # Construct augmented matrix for matrix exponential
    augmented_matrix = [
        A B; 
        zeros(size(B, 2), size(A, 1)) zeros(size(B, 2), size(B, 2))
    ]

    # Compute matrix exponential
    exp_matrix = exp(augmented_matrix * h)

    # Extract discrete LTI system matrices
    A_h = exp_matrix[1:size(A, 1), 1:size(A, 2)]
    B_h = exp_matrix[1:size(A, 1), size(A, 2)+1:end]

    return A_h, B_h
end

# Extract discrete LTI system matrices
A_cts = [0.0 1.0; -1.0 -0.1]
B_cts = [0.0; 1.0]
h = 0.1  # Time step
A, B = continuous_to_discrete(A_cts, B_cts, h)

- Let's create a 2D state.
\begin{equation}
    z = \begin{bmatrix} x \\ \dot{x} \\ y \\ \dot{y} \end{bmatrix}
\end{equation}

In [ ]:
Axy = I(2) ⊗ A
Bxy = [B zeros(2); zeros(2) B]

C = hcat([Axy^n * Bxy for n in 0:size(Axy, 1)-1]...)

In [ ]:
rank(C)

In [ ]:
Axy = I(2) ⊗ A

# Directly move the X particle only
Bxy = [[1; 0] zeros(2); [0; 1] zeros(2)]

C = hcat([Axy^n * Bxy for n in 0:size(Axy, 1)-1]...)

In [ ]:
rank(C)

What about quantum systems? They are nonlinear.

- BCH
\begin{equation}
    e^{X} e^{Y} = e^{X + Y + \tfrac{1}{2} [X, Y] + \tfrac{1}{12} ([X, [X, Y]] + [Y, [Y, X]])} 
\end{equation}

- Collect commutators, forming the **Dynamical Lie algebra**.

- Quick check: How can we test for linear dependence?

In [ ]:
# Linearly dependent: QR decomposition has a zero on diagonal 
H_drives = [PAULIS.X, PAULIS.Y]
# H_drives = [PAULIS.X, PAULIS.X]

M = stack(vec.(H_drives))
qr(M).R

- Quick check: What about systems with drift?

# II. Demos
-----

**Bloch sphere**

In [ ]:
Δ = 0.2
qubit = QuantumSystem(Δ * PAULIS.Z, [PAULIS.X, PAULIS.Y])
ψ0 = ket_from_string("e+g", [2])
ψT = ket_from_bitstring("0")
T = 50
Δt = 0.2
prob = QuantumStateSmoothPulseProblem(qubit, ψ0, ψT, T, Δt)

In [ ]:
fig, = plot_bloch(prob.trajectory)
fig

In [ ]:
plot(prob.trajectory, :a)

In [ ]:
solve!(prob, max_iter=100, options=IpoptOptions(eval_hessian=true))

In [ ]:
fig, = plot_bloch(prob.trajectory)
fig

In [ ]:
plot(prob.trajectory, :a)

**CZ gate**

\begin{equation}
    H(t) = \sum_j \tfrac{\eta}{2} a_j^\dagger a_j^\dagger a_j a_j + \Delta(t) (a_1^\dagger a_1) + g(t) (a_1^\dagger a_2 + a_1 a_2^\dagger)
\end{equation} 

In [ ]:
n_levels = 2
a = lift_operator(annihilate(n_levels), 1, 2)
b = lift_operator(annihilate(n_levels), 2, 2)
η = -0.3

H_drift = η/2 * (a'a'*a*a + b'b'*b*b)
H_drives = [a'a', (a'b + a*b')]
transmons = QuantumSystem(H_drift, H_drives)

U_goal = EmbeddedOperator(
    GATES.CZ, 
    get_subspace_indices([1:2, 1:2], [n_levels, n_levels]), 
    [n_levels, n_levels]
)

In [ ]:
commutator(A, B) = A * B - B * A

In [ ]:
A = H_drives[1]
B = H_drives[2]
commutator(A, B) |> sparse

In [ ]:
is_reachable(U_goal, transmons)

**Continous-variable quantum computing**

\begin{equation}
    H(t) = \Omega(t) a^\dagger + \Omega(t) a + \kappa(t) a^2 + \kappa^*(t) (a^\dagger)^2
\end{equation}

In [ ]:
n_levels = 5

a = annihilate(n_levels)
X = a + a'
Y = -im * (a - a')
X2 = a^2 + (a')^2
Y2 = -im * (a^2 - (a')^2)

Ω = 1.0
κ = 0.1

sys = QuantumSystem([X, Y, X2, Y2])

# Displacement and squeezing operators
function displacement(α)
    return exp(α * a' - conj(α) * a)
end

function squeezing(r)
    return exp((r / 2) * (a^2 - (a')^2))
end

# Initial states
ψ0 = I(n_levels)[:, 1] .+ 0.0im
ψα = displacement(im * 1.5) * ψ0;
ψs = squeezing(0.5) * displacement(0.5 + 0.5im) * ψ0;

## Coherent state

In [ ]:
T = 50
Δt = 0.4
prob = QuantumStateSmoothPulseProblem(sys, ψ0, ψα, T, Δt, dda_bound=0.1)
solve!(prob, max_iter=75, options=IpoptOptions(eval_hessian=true))

In [ ]:
rollout_fidelity(prob.trajectory, sys)

In [ ]:
fig, = plot_wigner(prob.trajectory, 1)
fig

In [ ]:
fig, = plot_wigner(prob.trajectory, prob.trajectory.T)
fig

In [ ]:
fig, = plot_wigner(QuantumObject(ψα))
fig

In [ ]:
plot(
    prob.trajectory, [:a],
    transformations = [:ψ̃ => ψ̃ -> abs2.(iso_to_ket(ψ̃)),],
    use_autolimits=true,
    transformation_titles = ["Population"],
)

### Minimum Time

In [ ]:
min_prob = QuantumStateMinimumTimeProblem(prob, ψα)

In [ ]:
solve!(min_prob, max_iter=100)

In [ ]:
fig, = plot_wigner(min_prob.trajectory, min_prob.trajectory.T)
fig

In [ ]:
plot(
    min_prob.trajectory, [:a],
    transformations = [:ψ̃ => ψ̃ -> abs2.(iso_to_ket(ψ̃)),],
    use_autolimits=true,
    transformation_titles = ["Population"],
)

## Squeezed states